### Manipulation of PEAKS de novo results of Trocas 8 (low water, November 2019) non-incubation samples LC-MS/MS data using python.

Starting with:

    PEAKS de novo results (.csv) of PTM-optimized sequencing >50% ALC

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 675) for another
#### 2. Change output file name in *IN 6*, *IN 7*, *IN 8*

In [1]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [2]:
    cd /home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/683_Trocas8_June2021_DENOVO_110/

/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/683_Trocas8_June2021_DENOVO_110


In [3]:
# read the CSVs into a dataframe using the read_csv function and call 'peaks'

peaks683 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/683_Trocas8_June2021_DENOVO_110/683_Trocas8_June2021_DENOVO_110_DN50.csv")

print("# redundant Peaks peptides >50% ALC in combined dataframe:", len(peaks683))

print(peaks683.columns)

# These columns mess things up- get rid of them

del peaks683['Fraction']
del peaks683['Scan']
del peaks683['Source File']
del peaks683['Tag Length']
del peaks683['PTM']
del peaks683['tag (>=0%)']
del peaks683['mode']
del peaks683['local confidence (%)']


columns = ['Peptide', 'ALC', 'length', 'm/z', 'z', 'RT', 'Area',
       'Mass', 'ppm']

peaks683.columns = columns

mean_len = peaks683['length'].mean()
print(mean_len)

# look at the dataframe
peaks683.head()

# redundant Peaks peptides >50% ALC in combined dataframe: 68
Index(['Fraction', 'Scan', 'Source File', 'Peptide', 'Tag Length', 'ALC (%)',
       'length', 'm/z', 'z', 'RT', 'Area', 'Mass', 'ppm', 'PTM',
       'local confidence (%)', 'tag (>=0%)', 'mode'],
      dtype='object')
9.352941176470589


,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm
0,LVTDLTK,98,7,395.2388,2,46.89,6280.0,788.4644,-1.7
1,LSSPATLNSR,98,10,523.2851,2,43.96,3940000.0,1044.5564,-0.7
2,LSSPATLNSR,98,10,523.2853,2,44.47,3940000.0,1044.5564,-0.4
3,VATVSPLR,95,8,421.7579,2,56.95,4280000.0,841.5021,-1.1
4,VATVSPLR,95,8,421.7580,2,57.45,4280000.0,841.5021,-0.8


The peptide column has the masses of modifications (e.g., 57.02 Da for carbamidomethylation of cysteine). We want to make new columns with all that information and make a column with only the 'stripped' peptide sequence that's just amino acids - this we can then align against other sequences, for example.

Modified residues were allowed for:

    fixed carbamodimethylation of cysteine 57.021464 C
    varialbe oxidation of methionine: 15.9949 M
    variable deamidation of asparagine, glumatine: 0.984016 NQ

We'll then write this manipulated dataframe to a new file.

In [4]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks683['A'] = peaks683['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks683['C'] = peaks683['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks683['D'] = peaks683['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks683['E'] = peaks683['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks683['F'] = peaks683['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks683['G'] = peaks683['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks683['H'] = peaks683['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks683 output, there will be no isoleucines (they're lumped in with leucines)
peaks683['I'] = peaks683['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks683['K'] = peaks683['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks683['L'] = peaks683['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks683['M'] = peaks683['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks683['N'] = peaks683['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks683['P'] = peaks683['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks683['Q'] = peaks683['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks683['R'] = peaks683['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks683['S'] = peaks683['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks683['T'] = peaks683['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks683['V'] = peaks683['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks683['W'] = peaks683['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks683['Y'] = peaks683['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks683['c-carb'] = peaks683['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks683['m-oxid'] = peaks683['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks683['n-deam'] = peaks683['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks683['q-deam'] = peaks683['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaks683['stripped_peptide'] = peaks683['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks683['stripped_length'] = peaks683['stripped_peptide'].apply(len)

# total the number of modifications in sequence
peaks683['ptm-total'] = peaks683['c-carb'] + peaks683['m-oxid'] + peaks683['n-deam'] + peaks683['q-deam']

# calculate NAAF numerator for each peptide k
peaks683['NAAF_num.'] = peaks683['Area'] / peaks683['stripped_length']

# write modified dataframe to new txt file, same name + 'stripped'
peaks683.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/675_BY_50_GF75_DN/683_BY_50_GF75_DN50.csv")

# check out the results
peaks683.head()

,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm,A,...,W,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,ptm-total,NAAF_num.
0,LVTDLTK,98,7,395.2388,2,46.89,6280.0,788.4644,-1.7,0,...,0,0,0,0,0,0,LVTDLTK,7,0,897.142857
1,LSSPATLNSR,98,10,523.2851,2,43.96,3940000.0,1044.5564,-0.7,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,394000.000000
2,LSSPATLNSR,98,10,523.2853,2,44.47,3940000.0,1044.5564,-0.4,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,394000.000000
3,VATVSPLR,95,8,421.7579,2,56.95,4280000.0,841.5021,-1.1,1,...,0,0,0,0,0,0,VATVSPLR,8,0,535000.000000
4,VATVSPLR,95,8,421.7580,2,57.45,4280000.0,841.5021,-0.8,1,...,0,0,0,0,0,0,VATVSPLR,8,0,535000.000000


In [5]:
# keep only stripped peptide I/L and NAAF
dn_683 = peaks683[['stripped_peptide', 'Area', 'NAAF_num.']]

dn_683.set_index('stripped_peptide')

# write modified dataframe to new txt file
dn_683.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/I-L_NAAFs/683_BY_50_GF75_DN50_ILnaafs.csv")

dn_683.head()

,stripped_peptide,Area,NAAF_num.
0,LVTDLTK,6280.0,897.142857
1,LSSPATLNSR,3940000.0,394000.000000
2,LSSPATLNSR,3940000.0,394000.000000
3,VATVSPLR,4280000.0,535000.000000
4,VATVSPLR,4280000.0,535000.000000


In [6]:
# made a new dataframe that contains the sums of certain columns 
# in the stripped peptide dataframe above (for >50% ALC)

index = ['sample total']

data = {'A': peaks683['A'].sum(),
        'C': peaks683['C'].sum(),
        'D': peaks683['D'].sum(),
        'E': peaks683['E'].sum(),
        'F': peaks683['F'].sum(),
        'G': peaks683['G'].sum(),
        'H': peaks683['H'].sum(),
        'I': peaks683['I'].sum(),
        'K': peaks683['K'].sum(),
        'L': peaks683['L'].sum(),
        'M': peaks683['M'].sum(),
        'N': peaks683['N'].sum(),
        'P': peaks683['P'].sum(),
        'Q': peaks683['Q'].sum(),
        'R': peaks683['R'].sum(),
        'S': peaks683['S'].sum(),
        'T': peaks683['T'].sum(),
        'V': peaks683['V'].sum(),
        'W': peaks683['W'].sum(),
        'Y': peaks683['Y'].sum(),
        'c-carb': peaks683['c-carb'].sum(),
        'm-oxid': peaks683['m-oxid'].sum(),
        'n-deam': peaks683['n-deam'].sum(),
        'q-deam': peaks683['q-deam'].sum(),
        'Total area': peaks683['Area'].sum(),
        'Total length': peaks683['stripped_length'].sum()
       }

totalpeaks683 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                            'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                            'c-carb', 'm-oxid', 'n-deam', 'q-deam', \
                                            'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks683['% C w/ carb'] = totalpeaks683['c-carb'] / totalpeaks683['C'] 

# calculate percentage of M's that are oxidized
totalpeaks683['% M w/ oxid'] = totalpeaks683['m-oxid'] / totalpeaks683['M'] 

# calculate percentage of N's that are deamidated
totalpeaks683['% N w/ deam'] = totalpeaks683['n-deam'] / totalpeaks683['N'] 

# calculate percentage of N's that are deamidated
totalpeaks683['% Q w/ deam'] = totalpeaks683['q-deam'] / totalpeaks683['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks683['NAAF denom.'] = totalpeaks683['Total area'] / totalpeaks683['Total length']

# write modified dataframe to new txt file
totalpeaks683.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/675_BY_50_GF75_DN/683_BY_50_GF75_DN50_totals.csv")

totalpeaks683.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,22,0,3,13,15,7,78,0,25,104,...,2,3,1,42506774.1,636,NaN,0.029851,0.230769,0.1,66834.550472


In [7]:
# use the calculated NAAF factor (in totalpeaks dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we would with Comet results

NAAF50 = 66834.550472

# use NAAF >50% ALC to get NAAF factor
peaks683['NAAF factor'] = (peaks683['NAAF_num.'])/NAAF50

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksAAPTM_683 = peaks683[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I', 'L', \
                                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', \
                                'n-deam', 'q-deam']].copy()

# multiply the NAAF50 factor by the AA total to normalize its abundance by peak area and peptide length

peaksAAPTM_683['A-NAAF50'] = peaksAAPTM_683['A'] * peaks683['NAAF factor']
peaksAAPTM_683['C-NAAF50'] = peaksAAPTM_683['C'] * peaks683['NAAF factor']
peaksAAPTM_683['D-NAAF50'] = peaksAAPTM_683['D'] * peaks683['NAAF factor']
peaksAAPTM_683['E-NAAF50'] = peaksAAPTM_683['E'] * peaks683['NAAF factor']
peaksAAPTM_683['F-NAAF50'] = peaksAAPTM_683['F'] * peaks683['NAAF factor']
peaksAAPTM_683['G-NAAF50'] = peaksAAPTM_683['G'] * peaks683['NAAF factor']
peaksAAPTM_683['H-NAAF50'] = peaksAAPTM_683['H'] * peaks683['NAAF factor']
peaksAAPTM_683['I-NAAF50'] = peaksAAPTM_683['I'] * peaks683['NAAF factor']
peaksAAPTM_683['K-NAAF50'] = peaksAAPTM_683['K'] * peaks683['NAAF factor']
peaksAAPTM_683['L-NAAF50'] = peaksAAPTM_683['L'] * peaks683['NAAF factor']
peaksAAPTM_683['M-NAAF50'] = peaksAAPTM_683['M'] * peaks683['NAAF factor']
peaksAAPTM_683['N-NAAF50'] = peaksAAPTM_683['N'] * peaks683['NAAF factor']
peaksAAPTM_683['P-NAAF50'] = peaksAAPTM_683['P'] * peaks683['NAAF factor']
peaksAAPTM_683['Q-NAAF50'] = peaksAAPTM_683['Q'] * peaks683['NAAF factor']
peaksAAPTM_683['R-NAAF50'] = peaksAAPTM_683['R'] * peaks683['NAAF factor']
peaksAAPTM_683['S-NAAF50'] = peaksAAPTM_683['S'] * peaks683['NAAF factor']
peaksAAPTM_683['T-NAAF50'] = peaksAAPTM_683['T'] * peaks683['NAAF factor']
peaksAAPTM_683['V-NAAF50'] = peaksAAPTM_683['V'] * peaks683['NAAF factor']
peaksAAPTM_683['W-NAAF50'] = peaksAAPTM_683['W'] * peaks683['NAAF factor']
peaksAAPTM_683['Y-NAAF50'] = peaksAAPTM_683['Y'] * peaks683['NAAF factor']

# multiply the NAAF50 factor by the PTMs normalize its abundance by peak area and peptide length

peaksAAPTM_683['ccarb-NAAF50'] = peaksAAPTM_683['c-carb'] * peaksAAPTM_683['NAAF factor']
peaksAAPTM_683['moxid-NAAF50'] = peaksAAPTM_683['m-oxid'] * peaksAAPTM_683['NAAF factor']
peaksAAPTM_683['ndeam-NAAF50'] = peaksAAPTM_683['n-deam'] * peaksAAPTM_683['NAAF factor']
peaksAAPTM_683['qdeam-NAAF50'] = peaksAAPTM_683['q-deam'] * peaksAAPTM_683['NAAF factor']


# write the dataframe to a new csv
peaksAAPTM_683.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/675_BY_50_GF75_DN/683_BY_50_GF75_DN50_naaf.csv")

peaksAAPTM_683.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,K,...,R-NAAF50,S-NAAF50,T-NAAF50,V-NAAF50,W-NAAF50,Y-NAAF50,ccarb-NAAF50,moxid-NAAF50,ndeam-NAAF50,qdeam-NAAF50
0,LVTDLTK,0.013423,0,0,1,0,0,0,0,1,...,0.000000,0.000000,0.026847,0.013423,0.0,0.0,0.0,0.0,0.0,0.0
1,LSSPATLNSR,5.895154,1,0,0,0,0,0,0,0,...,5.895154,17.685463,5.895154,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,LSSPATLNSR,5.895154,1,0,0,0,0,0,0,0,...,5.895154,17.685463,5.895154,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,VATVSPLR,8.004842,1,0,0,0,0,0,0,0,...,8.004842,8.004842,8.004842,16.009684,0.0,0.0,0.0,0.0,0.0,0.0
4,VATVSPLR,8.004842,1,0,0,0,0,0,0,0,...,8.004842,8.004842,8.004842,16.009684,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# made a dataframe that's the sum of NAAF corrected AAs and PTMs

index = ['sample total']

data = {'NAAF': peaksAAPTM_683['NAAF factor'].sum(),
        'A': peaksAAPTM_683['A-NAAF50'].sum(),
        'C': peaksAAPTM_683['C-NAAF50'].sum(),
        'D': peaksAAPTM_683['D-NAAF50'].sum(),
        'E': peaksAAPTM_683['E-NAAF50'].sum(),
        'F': peaksAAPTM_683['F-NAAF50'].sum(),
        'G': peaksAAPTM_683['G-NAAF50'].sum(),
        'H': peaksAAPTM_683['H-NAAF50'].sum(),
        'I': peaksAAPTM_683['I-NAAF50'].sum(),
        'K': peaksAAPTM_683['K-NAAF50'].sum(),
        'L': peaksAAPTM_683['L-NAAF50'].sum(),
        'M': peaksAAPTM_683['M-NAAF50'].sum(),
        'N': peaksAAPTM_683['N-NAAF50'].sum(),
        'P': peaksAAPTM_683['P-NAAF50'].sum(),
        'Q': peaksAAPTM_683['Q-NAAF50'].sum(),
        'R': peaksAAPTM_683['R-NAAF50'].sum(),
        'S': peaksAAPTM_683['S-NAAF50'].sum(),
        'T': peaksAAPTM_683['T-NAAF50'].sum(),
        'V': peaksAAPTM_683['V-NAAF50'].sum(),
        'W': peaksAAPTM_683['W-NAAF50'].sum(),
        'Y': peaksAAPTM_683['Y-NAAF50'].sum(),
        'c-carb': peaksAAPTM_683['ccarb-NAAF50'].sum(),
        'm-oxid': peaksAAPTM_683['moxid-NAAF50'].sum(),
        'n-deam': peaksAAPTM_683['ndeam-NAAF50'].sum(),
        'q-deam': peaksAAPTM_683['qdeam-NAAF50'].sum(),
       }

totalpeaks50_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', \
                                           'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', \
                                           'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaks50_NAAF['% C w/ carb.'] = totalpeaks50_NAAF['c-carb'] / totalpeaks50_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaks50_NAAF['% M w/ oxid'] = totalpeaks50_NAAF['m-oxid'] / totalpeaks50_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% N w/ deam'] = totalpeaks50_NAAF['n-deam'] / totalpeaks50_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% Q w/ deam'] = totalpeaks50_NAAF['q-deam'] / totalpeaks50_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks50_NAAF['NAAF check'] = totalpeaks50_NAAF['NAAF'] / 66834.550472

# write modified dataframe to new txt file, same name + totals
totalpeaks50_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/675_BY_50_GF75_DN/683_BY_50_GF75_DN50_naaf_totals.csv")

totalpeaks50_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,73.859448,55.62498,0.0,0.334664,0.32849,0.783945,6.093901,9.858719,0.0,7.93608,...,5.395805,0.0,0.045282,0.265955,0.080946,NaN,0.004554,0.014612,0.141225,0.001105


## Export stripped peptides >50% ALC

In [9]:
##### keep only stripped peptide column 
pep50 = peaks683[["stripped_peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep50.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/675_BY_50_GF75_DN/683_BY_50_GF75_DN50_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/675_BY_50_GF75_DN/683_BY_50_GF75_DN50_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/675_BY_50_GF75_DN/683_BY_50_GF75_DN50_stripped_peptides.fas

# removing redundancy
peaks50dedup = pd.DataFrame.drop_duplicates(pep50)

print("# redundant stripped Peaks peptides >50% ALC", len(pep50))
print("# nonredundant stripped Peaks peptides >50% ALC", len(peaks50dedup))
print("average peptide length Peaks peptides >50% ALC", peaks683['stripped_length'].mean())

# count all unique peptide (modified peptides included)
# keep only peptide column >50% ALC
pep50m = peaks683[["Peptide"]]

# deduplicate
pep50mdedup = pd.DataFrame.drop_duplicates(pep50m)

print("# redundant Peaks peptides >50% ALC", len(pep50m))
print("# nonredundant Peaks peptides", len(pep50mdedup))

# check
pep50.head()

# redundant stripped Peaks peptides >50% ALC 68
# nonredundant stripped Peaks peptides >50% ALC 37
average peptide length Peaks peptides >50% ALC 9.352941176470589
# redundant Peaks peptides >50% ALC 68
# nonredundant Peaks peptides 37


,stripped_peptide
0,LVTDLTK
1,LSSPATLNSR
2,LSSPATLNSR
3,VATVSPLR
4,VATVSPLR
